In [ ]:
from ROOT import TString, FairRunAna, FairFileSource, FairParRootFileIo, FairParAsciiFileIo
from ROOT import RhoMassParticleSelector, PndAnalysis, TFile,  RhoCandList, PndAnalysis
import numpy as np


prefix = "signal"
nevts = 0

OutFile = "Out_dummy.root"

inPidFile = prefix+"_pid.root"
inParFile = prefix+"_par.root"
pidParFile = TString('/home/lewis/fairsoft_may18/pandaroot/macro/params/all.par')

In [ ]:
fRun = FairRunAna()
rtdb = fRun.GetRuntimeDb()
source = FairFileSource(inPidFile)
source.AddFriend("signal_full_nn.root")
fRun.SetSource(source)

parIO = FairParRootFileIo()
parIO.open(inParFile)
parIOPid = FairParAsciiFileIo()
parIOPid.open(pidParFile.Data(),"in")

rtdb.setFirstInput(parIO)
rtdb.setSecondInput(parIOPid)
rtdb.setOutput(parIO)

fRun.SetOutputFile(OutFile)
fRun.Init()
out = TFile.Open(prefix+"_conventional","RECREATE")

In [ ]:
theAnalysis = PndAnalysis()
if (nevts==0):
    nevts= theAnalysis.GetEntries()

muon = RhoCandList()
pion = RhoCandList()

In [ ]:
i = 0
num_classes = 5
y_test = []
y_score = []

# iterate through each event for analysis
while theAnalysis.GetEvent():
    i += 1
    # print a statement for every 10,000 particles
    if ((i%10000)==0):
        print("evt ",i)
        
    
    #choose which particle, cut, and method to perform the analysis
    #theAnalysis.FillList(muon,  "MuonAll",  "PidAlgoMvd;PidAlgoStt;PidAlgoDrc;PidAlgoMdtHardCuts;PidAlgoDisc;PidAlgoEmcBayes;PidAlgoSciT;PidAlgoRich")
    theAnalysis.FillList(muon,  "MuonAll",  "PidAlgoMl")
    for ii in range(muon.GetLength()):
        # only calculate the probability for the monte carlo truth matched particles
        if not (theAnalysis.McTruthMatch(muon[ii])):
            prob = np.empty(num_classes)
            for jj in range(num_classes):
                prob[jj] = (muon[ii].GetPidInfo(jj))
            y_score.append(prob)    # append the probability to the y_score array
            y_test.append(2)        # append the true value of the muon to the array
    
    
    #choose which particle, cut, and method to perform the analysis
    #theAnalysis.FillList(pion,  "PionAll",  "PidAlgoMvd;PidAlgoStt;PidAlgoDrc;PidAlgoMdtHardCuts;PidAlgoDisc;PidAlgoEmcBayes;PidAlgoSciT;PidAlgoRich")
    theAnalysis.FillList(pion,  "PionAll",  "PidAlgoMl")
    for ii in range(pion.GetLength()):
        # only calculate the probability for the monte carlo truth matched particles
        if not (theAnalysis.McTruthMatch(pion[ii])): 
            prob = np.empty(num_classes)
            for jj in range(num_classes):
                prob[jj] = (pion[ii].GetPidInfo(jj))
            y_score.append(prob)    # append the probability to the y_score array
            y_test.append(1)        # append the true value of the pion to the array


# turn the values into a numpy array and remove any NaN values
y_test = np.asarray(y_test)
y_score = np.asarray(y_score)
y_score = np.nan_to_num(y_score,0.0)

In [ ]:
# print the arrays
# y_test holds each particles true value
# y_score holds 5 different particle probabilities for every particle
print(y_test)
print(y_score)

In [ ]:
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4])
n_classes = y_test.shape[1]
import matplotlib.pyplot as plt
%matplotlib tk

# set some variables that wil be used for plotting
lw=2
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

from itertools import cycle


#plot the roc curve for the muons and pions

#plt.plot(fpr[0], tpr[0], color='aqua', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$e^{-}$',roc_auc[0]))
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\pi^{-}$',roc_auc[1]))
plt.plot(fpr[2], tpr[2], color='cornflowerblue', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\mu^{-}$',roc_auc[2]))
#plt.plot(fpr[3], tpr[3], color='black', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$k^{-}$',roc_auc[3]))
#plt.plot(fpr[4], tpr[4], color='green', lw=lw, label='ROC for {0}   (area = {1:0.2f})'.format('$\overline{p}$',roc_auc[4]))

plt.ylim([0.0, 1.0])
plt.xlim([0.0, 1.0])
plt.xlabel('False Positive Rate', fontsize=22)
plt.ylabel('True Positive Rate', fontsize=22)
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right', fontsize=10)
plt.savefig('full_roc_curve_nn.png')
plt.show()

In [ ]:
# Compute Precision Recall curve for each class
from sklearn.metrics import precision_recall_curve

fpr = dict()
tpr = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = precision_recall_curve(y_test[:, i], y_score[:, i])

    
#plot the preicion recall curve for the muons and pions

#plt.plot(fpr[0], tpr[0], color='aqua', lw=lw, label='Electrons')
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='Pions')
plt.plot(fpr[2], tpr[2], color='cornflowerblue', lw=lw, label='Muons')
#plt.plot(fpr[3], tpr[3], color='black', lw=lw, label='Kaons')
#plt.plot(fpr[4], tpr[4], color='green', lw=lw, label='Protons')
plt.xlim([0.0,1.0])
plt.ylim([0.0, 1.0])

plt.xlabel('Recall', fontsize=22)
plt.ylabel('Precision', fontsize=22)
plt.legend(loc='lower right', fontsize=10)
plt.savefig('precision_recall_curve_nn.png')